In [1]:
import json
import pandas as pd
from tabulate import tabulate

# Uthmani and indopak init print

In [ ]:
uthmani_path = "data/uthmani.json"
indopak_path = "data/indopak.json"

uthmani_df = pd.read_json(uthmani_path, encoding="utf-8")
indopak_df = pd.read_json(indopak_path, encoding="utf-8")

In [19]:
print("source indopak shape:", indopak_df.shape)
print("source uthmani shape:", uthmani_df.shape)

# Transposition needed

indopak_transpose = indopak_df.transpose()
print("indopak transpose shape:", indopak_transpose.shape)
uthmani_transpose = uthmani_df.transpose()
print("uthmani transpose shape:", uthmani_transpose.shape)

print("Uthmani DataFrame:")
print(tabulate(uthmani_transpose.head(), headers='keys', tablefmt='psql'))

print("\nIndopak DataFrame:")
print(tabulate(indopak_transpose.head(), headers='keys', tablefmt='psql'))

source indopak shape: (5, 6236)
source uthmani shape: (6, 83668)
indopak transpose shape: (6236, 5)
uthmani transpose shape: (83668, 6)
Uthmani DataFrame:
+-------+------+---------+--------+--------+------------+---------+
|       |   id |   surah |   ayah |   word | location   | text    |
|-------+------+---------+--------+--------+------------+---------|
| 1:1:1 |    1 |       1 |      1 |      1 | 1:1:1      | بِسْمِ     |
| 1:1:2 |    2 |       1 |      1 |      2 | 1:1:2      | ٱللَّهِ    |
| 1:1:3 |    3 |       1 |      1 |      3 | 1:1:3      | ٱلرَّحْمَـٰنِ |
| 1:1:4 |    4 |       1 |      1 |      4 | 1:1:4      | ٱلرَّحِيمِ  |
| 1:1:5 |    5 |       1 |      1 |      5 | 1:1:5      | ١       |
+-------+------+---------+--------+--------+------------+---------+

Indopak DataFrame:
+-----+------+-------------+---------+--------+------------------------+
|     |   id | verse_key   |   surah |   ayah | text                   |
|-----+------+-------------+---------+--------+----

In [12]:
df = pd.read_csv("data/quran-md-indopak-joined.csv", encoding="utf-8-sig")

print("arabic text comparison")
print(df.head())

arabic text comparison
   surah_id  ayah_id surah_name_ar surah_name_en surah_name_tr  ayah_count  \
0         1        1       الفاتحة   The Opening    Al-Faatiha           7   
1         1        2       الفاتحة   The Opening    Al-Faatiha           7   
2         1        3       الفاتحة   The Opening    Al-Faatiha           7   
3         1        4       الفاتحة   The Opening    Al-Faatiha           7   
4         1        5       الفاتحة   The Opening    Al-Faatiha           7   

                                             ayah_en  \
0  In the name of Allah, the Entirely Merciful, t...   
1  [All] praise is [due] to Allah, Lord of the wo...   
2    The Entirely Merciful, the Especially Merciful,   
3                Sovereign of the Day of Recompense.   
4      It is You we worship and You we ask for help.   

                              ayah_tr  \
0    bis'mi l-lahi l-raḥmāni l-raḥīmi   
1   al-ḥamdu lillahi rabbi l-ʿālamīna   
2                 al-raḥmāni l-raḥīmi   
3      